## Try with chains

In [95]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///test2.db")
print(db.dialect)
print(db.get_usable_table_names())
# db.run("select Female_ from Options where Demographics_='Always_' and Question_Number=3;")

sqlite
['Question_1', 'Question_10', 'Question_11', 'Question_12', 'Question_13', 'Question_14', 'Question_15', 'Question_16', 'Question_17', 'Question_18', 'Question_19', 'Question_2', 'Question_20', 'Question_3', 'Question_4', 'Question_5', 'Question_6', 'Question_7', 'Question_8', 'Question_9', 'Questions']


In [96]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from dotenv import load_dotenv
load_dotenv()
llm =  ChatOpenAI(model="gpt-4o", temperature=0)



chain = create_sql_query_chain(llm, db)
op_cols = db.run("""
    SELECT Options from "Question_1";
""")
ques = db.run(
    """
    SELECT * from Questions;
"""
)
res = chain.invoke({"question": f" Option column has values : {op_cols} And Questions table contain {ques} from survey which are linked with all other tables. Keeping that in mind How much male buy sustainable and enviromentaly friendly products? only return query"})
print(res)

```sql
SELECT "Male" 
FROM "Question_4" 
WHERE "Options" = 'Male'
LIMIT 5;
```


In [94]:
db.run("""
      SELECT "Male" 
FROM "Question_4" 
WHERE "Options" = 'Male'
LIMIT 5;
       """)

"[('Male',)]"

In [89]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

# chain.invoke({"question": "Option column has values : {op_cols} And Questions table contain :{ques}. Keeping that in mind What is the most important issue in London ?"})
# response
res = chain.invoke({"question": f" Option column has values : {op_cols} And Questions table contain {ques} from survey which are linked with all other tables. Keeping that in mind How much male buy sustainable and enviromentaly friendly products? only return query"})
print(res)

Error: (sqlite3.OperationalError) near "```sql
SELECT "Male" 
FROM "Question_4" 
WHERE "Options" = 'Male'
LIMIT 5;
```": syntax error
[SQL: ```sql
SELECT "Male" 
FROM "Question_4" 
WHERE "Options" = 'Male'
LIMIT 5;
```]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [90]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

res = chain.invoke({"question": f" Option column has values : {op_cols} And Questions table contain {ques} from survey which are linked with all other tables. Keeping that in mind How much male buy sustainable and enviromentaly friendly products? only return query"})
print(res)

The SQL query provided in the question is incorrect and results in a syntax error. To determine how much males buy sustainable and environmentally friendly products, we need to construct a query that correctly references the relevant tables and columns. Based on the information provided, it seems like the "Question_4" table is related to the question about the importance of sustainability credentials of brands/products. However, the query should be structured to extract relevant data about male respondents and their purchasing behavior regarding sustainable products.

Here is a corrected version of the SQL query that could be used to find out how many males buy sustainable and environmentally friendly products:

```sql
SELECT COUNT(*)
FROM "Question_4" AS q4
JOIN "Responses" AS r ON q4.response_id = r.id
JOIN "Demographics" AS d ON r.demographic_id = d.id
WHERE d.gender = 'Male' AND q4.answer IN ('Always', 'Often', 'Sometimes');
```

This query assumes the existence of a "Responses" ta

In [91]:
db.run(
    """
    SELECT COUNT(*)
FROM "Question_4" AS q4
JOIN "Responses" AS r ON q4.response_id = r.id
JOIN "Demographics" AS d ON r.demographic_id = d.id
WHERE d.gender = 'Male' AND q4.answer IN ('Always', 'Often', 'Sometimes');
"""
)

OperationalError: (sqlite3.OperationalError) no such table: Responses
[SQL: 
    SELECT COUNT(*)
FROM "Question_4" AS q4
JOIN "Responses" AS r ON q4.response_id = r.id
JOIN "Demographics" AS d ON r.demographic_id = d.id
WHERE d.gender = 'Male' AND q4.answer IN ('Always', 'Often', 'Sometimes');
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

### Code to use

In [111]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

answer = agent_executor.invoke(
    {
        "input": f" Option column has values : {op_cols} And Questions table contain {ques} from survey which are linked with all other tables. Keeping that in mind How much male buy sustainable and enviromentaly friendly products?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Question_1, Question_10, Question_11, Question_12, Question_13, Question_14, Question_15, Question_16, Question_17, Question_18, Question_19, Question_2, Question_20, Question_3, Question_4, Question_5, Question_6, Question_7, Question_8, Question_9, Questions
Invoking: `sql_db_schema` with `{'table_names': 'Question_3, Question_4, Question_5, Question_6, Question_7, Questions'}`



CREATE TABLE "Question_3" (
	"Vegetarian " TEXT, 
	"Vegan " TEXT, 
	"Pescatarian " TEXT, 
	"Halal " TEXT, 
	"Kosher " TEXT, 
	"None of the above " TEXT, 
	"Question_Number" TEXT, 
	"Options" TEXT
)

/*
3 rows from Question_3 table:
Vegetarian 	Vegan 	Pescatarian 	Halal 	Kosher 	None of the above 	Question_Number	Options
90.0	44.0	61.0	87.0	11.0	715.0	0.0	Value
41.0	18.0	29.0	47.0	6.0	335.0	0.0	Male
49.0	26.0	32.0	40.0	5.0	380.0	0.0	Female
*/


CREATE TABLE "Question_4" (
	"Always " TEXT, 
	"Often " TEXT, 
	"Sometimes " 

In [112]:
answer

{'input': ' Option column has values : [(\'Value\',), (\'Male\',), (\'Female\',), (\'North East\',), (\'North West\',), (\'Yorkshire & The Humber\',), (\'East Midlands\',), (\'West Midlands\',), (\'East of England\',), (\'London\',), (\'South East\',), (\'South West\',), (\'Wales\',), (\'Scotland\',), (\'Northern Ireland\',), (\'18-24\',), (\'25-34\',), (\'35-44\',), (\'45-54\',), (\'55-64\',), (\'Vegetarian\',), (\'Vegan\',), (\'Pescatarian\',), (\'Halal\',), (\'Kosher\',), (\'None of the above\',), (\'Always\',), (\'Often\',), (\'Sometimes\',), (\'Rarely\',), (\'Never\',), (\'Very important\',), (\'Fairly important\',), (\'Neither\',), (\'Fairly unimportant\',), (\'Very unimportant\',), (\'Environmentally Friendly\',), (\'Good for the planet\',), (\'Less waste\',), (\'Local\',), (\'Ethical\',), ("Don\'t Know",), (\'Longer lasting\',), (\'No natural resource depletion\',), (\'Recyclable\',), (\'Locally Sourced\',), (\'Zero Carbon\',), (\'Reuse or Repurpose\',), (\'No Group\',), (\'The

In [118]:
query_gen_system = PromptTemplate.from_template("""
    You are a Business Analyst And you have recieved data from SQL agent. Given the following user question, and result, answer the user question. 
    question: {input}
    result: {output} 
    Answer:""")


chain = agent_executor | query_gen_system | llm | StrOutputParser()
res = chain.invoke(
    {
        "input": f" Option column has values : {op_cols} And Questions table contain {ques} from survey which are linked with all other tables. Keeping that in mind How much male buy sustainable and enviromentaly friendly products?"
    }
)
print(res)




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Question_1, Question_10, Question_11, Question_12, Question_13, Question_14, Question_15, Question_16, Question_17, Question_18, Question_19, Question_2, Question_20, Question_3, Question_4, Question_5, Question_6, Question_7, Question_8, Question_9, Questions
Invoking: `sql_db_schema` with `{'table_names': 'Question_3, Question_4, Question_5, Question_6, Question_7, Questions'}`



CREATE TABLE "Question_3" (
	"Vegetarian " TEXT, 
	"Vegan " TEXT, 
	"Pescatarian " TEXT, 
	"Halal " TEXT, 
	"Kosher " TEXT, 
	"None of the above " TEXT, 
	"Question_Number" TEXT, 
	"Options" TEXT
)

/*
3 rows from Question_3 table:
Vegetarian 	Vegan 	Pescatarian 	Halal 	Kosher 	None of the above 	Question_Number	Options
90.0	44.0	61.0	87.0	11.0	715.0	0.0	Value
41.0	18.0	29.0	47.0	6.0	335.0	0.0	Male
49.0	26.0	32.0	40.0	5.0	380.0	0.0	Female
*/


CREATE TABLE "Question_4" (
	"Always " TEXT, 
	"Often " TEXT, 
	"Sometimes " 

In [119]:
print(res)

The data indicates that a significant number of males purchase sustainable and environmentally friendly products with varying frequency. Specifically:

- 90 males always purchase such products.
- 161 males often purchase them.
- 179 males sometimes purchase them.
- 34 males rarely purchase them.
- 13 males never purchase them.

This suggests that while there is a strong inclination among males to buy sustainable products, the frequency varies, with the majority falling into the "often" and "sometimes" categories.
